In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

## Experiment 1: Train Logistic Regression Classifier

In [4]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.96      0.95       270
           1       0.60      0.50      0.55        30

    accuracy                           0.92       300
   macro avg       0.77      0.73      0.75       300
weighted avg       0.91      0.92      0.91       300



## Experiment 2: Train Random Forest Classifier

In [5]:
rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       270
           1       0.95      0.63      0.76        30

    accuracy                           0.96       300
   macro avg       0.96      0.81      0.87       300
weighted avg       0.96      0.96      0.96       300



## Experiment 3: Train XGBoost

In [6]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       270
           1       0.96      0.80      0.87        30

    accuracy                           0.98       300
   macro avg       0.97      0.90      0.93       300
weighted avg       0.98      0.98      0.98       300



## Experiment 4: Handle class imbalance using SMOTETomek and then Train XGBoost

In [7]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [8]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

In [9]:
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_res, y_train_res)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       270
           1       0.81      0.83      0.82        30

    accuracy                           0.96       300
   macro avg       0.89      0.91      0.90       300
weighted avg       0.96      0.96      0.96       300



## Track Experiments Using MLFlow

In [10]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [11]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]

    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)


In [12]:
reports

[{'0': {'precision': 0.9454545454545454,
   'recall': 0.9629629629629629,
   'f1-score': 0.9541284403669725,
   'support': 270.0},
  '1': {'precision': 0.6,
   'recall': 0.5,
   'f1-score': 0.5454545454545454,
   'support': 30.0},
  'accuracy': 0.9166666666666666,
  'macro avg': {'precision': 0.7727272727272727,
   'recall': 0.7314814814814814,
   'f1-score': 0.749791492910759,
   'support': 300.0},
  'weighted avg': {'precision': 0.9109090909090909,
   'recall': 0.9166666666666666,
   'f1-score': 0.91326105087573,
   'support': 300.0}},
 {'0': {'precision': 0.9676258992805755,
   'recall': 0.9962962962962963,
   'f1-score': 0.9817518248175182,
   'support': 270.0},
  '1': {'precision': 0.9545454545454546,
   'recall': 0.7,
   'f1-score': 0.8076923076923077,
   'support': 30.0},
  'accuracy': 0.9666666666666667,
  'macro avg': {'precision': 0.961085676913015,
   'recall': 0.8481481481481481,
   'f1-score': 0.8947220662549129,
   'support': 300.0},
  'weighted avg': {'precision': 0.9663

In [13]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [15]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detection V2")
mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metric('accuracy', report['accuracy'])
        mlflow.log_metric('recall_class_1', report['1']['recall'])
        mlflow.log_metric('recall_class_0', report['0']['recall'])
        mlflow.log_metric('f1_score_macro', report['macro avg']['f1-score'])        
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/01/12 19:49:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/569532407459835564/runs/967363092b6d483cb5ffcce2254bdd44
🧪 View experiment at: http://localhost:5000/#/experiments/569532407459835564


2025/01/12 19:49:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/569532407459835564/runs/fdf18659548944eaa99272b7679de084
🧪 View experiment at: http://localhost:5000/#/experiments/569532407459835564


2025/01/12 19:49:26 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\USER\AppData\Local\Temp\tmpdi9gwnal\model, flavor: xgboost). Fall back to return ['xgboost==2.1.3']. Set logging level to DEBUG to see the full traceback. 
2025/01/12 19:49:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/569532407459835564/runs/0c1ce54b09974a82bde1bcfcd92c6f19
🧪 View experiment at: http://localhost:5000/#/experiments/569532407459835564


2025/01/12 19:49:28 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\USER\AppData\Local\Temp\tmprjcidagg\model, flavor: xgboost). Fall back to return ['xgboost==2.1.3']. Set logging level to DEBUG to see the full traceback. 
2025/01/12 19:49:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/569532407459835564/runs/a18dbc575368481dad2698f9f66c7b2f
🧪 View experiment at: http://localhost:5000/#/experiments/569532407459835564


## Register the model

In [16]:
model_name = 'XGB-Smote'
run_id='a18dbc575368481dad2698f9f66c7b2f'
model_uri = f'runs:/{run_id}/model_name'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

Registered model 'XGB-Smote' already exists. Creating a new version of this model...
2025/01/12 19:49:43 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 1


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/569532407459835564/runs/a18dbc575368481dad2698f9f66c7b2f
🧪 View experiment at: http://localhost:5000/#/experiments/569532407459835564


Created version '1' of model 'XGB-Smote'.
